### Indexing: Creating a Chroma Vectorstore

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import pandas as pd

In [3]:
files = pd.read_csv("course_section_descriptions.csv", encoding='ANSI')

## Data preprocessing

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Assuming 'files' DataFrame is already defined and includes necessary columns

# Define weights for different text components
weight_course_name = 5
weight_section_name = 3
weight_section_description = 2
weight_other = 1  # For other components like course technology and course description

# Create a unique identifier for each section combining course_id and section_id
files['unique_id'] = files['course_id'].astype(str) + '-' + files['section_id'].astype(str)

# Create metadata for each section
files['metadata'] = files.apply(lambda row: {
    "course_name": row['course_name'],
    "section_name": row['section_name'],
    "section_description": row['section_description']
}, axis=1)



C:\Users\nikol\AppData\Local\Continuum\anaconda3\envs\langchain_env_24\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def create_embeddings(row):
    # Encode individual components
    emb_course_name = model.encode(row['course_name'], show_progress_bar=False) * weight_course_name
    emb_section_name = model.encode(row['section_name'], show_progress_bar=False) * weight_section_name
    emb_section_description = model.encode(row['section_description'], show_progress_bar=False) * weight_section_description
    emb_course_tech = model.encode(row['course_technology'], show_progress_bar=False) * weight_other
    emb_course_desc = model.encode(row['course_description'], show_progress_bar=False) * weight_other

    # Combine embeddings by averaging them
    combined_embedding = (emb_course_name + emb_section_name + emb_section_description + emb_course_tech + emb_course_desc) / (weight_course_name + weight_section_name + weight_section_description + 2 * weight_other)
    return combined_embedding

# Initialize the model
model = SentenceTransformer('multi-qa-distilbert-cos-v1')

# Apply the function to create weighted embeddings
files['embedding'] = files.apply(create_embeddings, axis=1)


## Embedding

## Connect to Pinecone Index

In [6]:
import os
from pinecone import Pinecone, ServerlessSpec

In [7]:
from dotenv import load_dotenv, find_dotenv

In [8]:
load_dotenv(find_dotenv(), override = True)

True

In [9]:
import pinecone
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

In [10]:
index = pc.Index("my-index")

In [11]:
index.delete(delete_all = True)

{}

In [12]:
# Prepare the vectors for upserting
vectors_to_upsert = [(row['unique_id'], row['embedding'].tolist(), row['metadata']) for index, row in files.iterrows()]




In [13]:

# Upsert data
index.upsert(vectors=vectors_to_upsert)

print("Data successfully upserted to Pinecone index.")

Data successfully upserted to Pinecone index.


In [14]:
# Ensure you've already initialized and configured Pinecone and the model
# If not, you need to run the initialization code provided earlier

# Create the query embedding
query = "lasso"
query_embedding = model.encode(query, show_progress_bar=False).tolist()





In [18]:
query_results = index.query(
   # namespace="my-index",
    vector=[query_embedding],
    top_k=15,
    include_metadata=True
)

In [21]:
import pprint

In [22]:
score_threshold = 0.3

In [35]:
# Assuming query_results are fetched and include metadata
for match in query_results['matches']:
    if match['score'] >= score_threshold:
        course_details = match.get('metadata', {})
        course_name = course_details.get('course_name', 'N/A')
        section_name = course_details.get('section_name', 'N/A')
        section_description = course_details.get('section_description', 'No description available')
        
        pprint.pprint(f"Matched item ID: {match['id']}, Score: {match['score']}")
        pprint.pprint(f"Course: {course_name}")
        pprint.pprint(f"Section: {section_name}, Description: {section_description}", width = 100) 
       #pprint.pprint() 

'Matched item ID: 56-492, Score: 0.619535148'
'Course: Machine Learning with Ridge and Lasso Regression'
('Section: Ridge and Lasso Regression – Practical Case, Description: In this section, we will walk '
 'you through the implementation of ridge and lasso regression using sk-learn in Python. We apply '
 'these methods to a real dataset in order to increase the performance of a regression algorithm '
 'by preventing overfitting. Furthermore, we demonstrate how regularization works and uncover the '
 'differences between ridge and lasso models.  ')
'Matched item ID: 56-490, Score: 0.574204624'
'Course: Machine Learning with Ridge and Lasso Regression'
('Section: Regularization Basics, Description: As an introduction to the course, we explore the '
 'concept of regularization and explain how it can be leveraged to prevent overfitting and '
 'multicollinearity issues. In addition, we demonstrate the theoretical differences between the '
 'mechanisms of ridge and lasso regression. ')
'Mat